<div align="center"><h1> Seismic Modelling </h1></div>

In this example we aim to highlight the core ideas behind seismic modelling, where we create a numerical model that modelling the simple wave equation. This model will then form the basis for further tutorials on the implementation.

![Seismic Modelling](./images/survey-ship-diagram.png)

## 1D Wave Equation

As with the Jacobi program, you don't need to really understand this algorithm to work with the code, but, we will take time here to introduce it.

The problem we'll solve is the 1D wave equation:

$$
\frac{\partial^2}{\partial t^2} u(x,t) = c^2 \frac{\partial^2}{\partial x^2} u(x,t)
$$

Here $u = u(x,t)$ is a scalar function of space ($x$) and time ($t$), and $c$ is a (constant) characteristic wave speed (for example, the speed of sound if the wave in question is propagating in air).

Implementing the centered-difference discretization of the spatial and time derivatives, one way to write this is:

$$
\frac{1}{\Delta t^2} \left(u_{i}^{n+1} - 2 u_{i}^{n} + u_{i}^{n-1}\right) = \frac{c^2}{\Delta x^2} \left(u_{i+1}^{n} - 2 u_{i}^{n} + u_{i-1}^{n}\right)
$$

Where subscripts denote spatial indices and superscripts denote timesteps. Rearranging this for the unknown $u_{i}^{n+1}$ in terms of known quantities at timesteps $n$ and $n-1$, we have:

$$
u_{i}^{n+1} = 2 u_{i}^{n} - u_{i}^{n-1} + \left(\frac{c\, \Delta t}{\Delta x}\right)^2 \left(u_{i+1}^{n} - 2 u_{i}^{n} + u_{i-1}^{n}\right)
$$

### Solving

To solve using this method, we simply need to retain the value of the solution at two previous timesteps, and then replace the old data after each update.

We're going to specify a 1D domain from $x = 0.0$ to $x = 1.0$, and discretize into $N$ points with a grid spacing of $\Delta x = 1.0\, /\, (N - 1)$. $\Delta t$ [must be chosen](https://en.wikipedia.org/wiki/Courant%E2%80%93Friedrichs%E2%80%93Lewy_condition) so that it is less than or equal to $c \Delta x$. To simplify, we'll choose $c = 1.0$ so that we don't have to worry about that term floating around.

We're going to specify that $u(0, t) = u(1, t) = 0$. We can think of this like we're solving a wave propagating in a string, where the two ends of the string are held taut. What we specify is the initial condition $u(x, 0)$, a simple sine wave, as well as an initial condition (the velocity at $t = 0$ is zero, which is effectively implemented by starting with $u^{n} == u^{n-1}$).

The period of this wave is 1.0, so after simulating up to $t = 1$, the wave should return exactly to where it started. We will use that to verify our solution -- the check at the end of the code will print an "error" which is the $L^2$ norm of the current solution with respect to the initial solution.

## ⊗ Sequential

This example is implemented in standard C++  for a CPU in [wave-sequential.cpp]. Take some time to review the algorithm and its parallel implementation, and examine the output below.

In [ ]:
%%writefile wave-sequential.cpp
#include <iostream>
#include <limits>
#include <cstdio>
#include <cmath>
#include <sys/time.h>

// Number of points in the overall spatial domain
#define NUM_POINTS 262144

void wave_update (float* u, const float* u_old, const float* u_older, float dtdxsq, int N)
{
  
    for(int idx = 1; idx < N - 1; idx++)
    {

        float u_old_right = u_old[idx+1];
        float u_old_left = u_old[idx-1];

        u[idx] = 2.0f * u_old[idx] - u_older[idx] +
                 dtdxsq * (u_old_right - 2.0f * u_old[idx] + u_old_left);
    }
}

void initialize (float* u, int N)
{

    for(int idx = 0; idx < N; idx++) 
    {
        u[idx] = std::sin(2.0f * M_PI * idx / static_cast<float>(NUM_POINTS - 1));
    }
}


int main(int argc, char **argv) 
{
    const int N = NUM_POINTS;

    // Allocate space for the grid data, and the temporary buffer
    // for the "old" and "older" data.
    float* u_older = (float*) malloc(sizeof(float) * N);
    float* u_old = (float*) malloc(sizeof(float) * N);
    float* u = (float*) malloc(sizeof(float) * N);

    initialize(u_older, N);
    initialize(u_old, N);
    initialize(u, N);
     
    // Now iterate until we've completed a full period
    const float period = 1.0f;
    const float start_time = 0.0f;
    const float stop_time = period;

    // Maximum stable timestep is <= dx
    float stability_factor = 0.5f;
    float dx = 1.0f / (NUM_POINTS - 1);
    float dt = stability_factor * dx;

    float t = start_time;
    const float safety_factor = (1.0f - 1.0e-5f);

    int num_steps = 0;

    // Start measuring time
    struct timeval begin, end;
    gettimeofday(&begin, 0);
    
    while (t < safety_factor * stop_time) {
        // Make sure the last step does not go over the target time
        if (t + dt >= stop_time) {
            dt = stop_time - t;
        }

        float dtdxsq = (dt / dx) * (dt / dx);

        // Launch kernel to do the calculation
        wave_update(u, u_old, u_older, dtdxsq, N);

        // Swap u_old and u_older
        std::swap(u_old, u_older);

        // Swap u and u_old
        std::swap(u, u_old);

        // Print out diagnostics periodically
        if (num_steps % 100000 == 0) {
            std::cout << "Current integration time = " << t << "\n";
        }

        // Update t
        t += dt;
        ++num_steps;
    }
    
    gettimeofday(&end, 0);
    long seconds = end.tv_sec - begin.tv_sec;
    long microseconds = end.tv_usec - begin.tv_usec;
    double elapsed = seconds + microseconds*1e-6;
    
    printf("\n%d  %.3f seconds\n", NUM_POINTS, elapsed);

    // Clean up
    free(u_older);
    free(u_old);
    free(u);

    return 0;
}

In [ ]:
!g++ wave-sequential.cpp -o wave-sequential -O3

In [ ]:
!./wave-sequential

## ⊗ OpenMP

In [ ]:
%%writefile wave-omp.cpp
#include <iostream>
#include <limits>
#include <cstdio>
#include <cmath>
#include <omp.h>
#include <sys/time.h>

// Number of points in the overall spatial domain
#define NUM_POINTS 262144

void wave_update (float* u, const float* u_old, const float* u_older, float dtdxsq, int N)
{
    int idx;
    
    #pragma omp parallel for private (idx)
    for(idx = 1; idx < N - 1; idx++)
    {

        float u_old_right = u_old[idx+1];
        float u_old_left = u_old[idx-1];

        u[idx] = 2.0f * u_old[idx] - u_older[idx] +
                 dtdxsq * (u_old_right - 2.0f * u_old[idx] + u_old_left);
    }
}

void initialize (float* u, int N)
{

    for(int idx = 0; idx < N; idx++) 
    {
        u[idx] = std::sin(2.0f * M_PI * idx / static_cast<float>(NUM_POINTS - 1));
    }
}


int main(int argc, char **argv) 
{
    const int N = NUM_POINTS;

    // Allocate space for the grid data, and the temporary buffer
    // for the "old" and "older" data.
    float* u_older = (float*) malloc(sizeof(float) * N);
    float* u_old = (float*) malloc(sizeof(float) * N);
    float* u = (float*) malloc(sizeof(float) * N);

    initialize(u_older, N);
    initialize(u_old, N);
    initialize(u, N);
     
    // Now iterate until we've completed a full period
    const float period = 1.0f;
    const float start_time = 0.0f;
    const float stop_time = period;

    // Maximum stable timestep is <= dx
    float stability_factor = 0.5f;
    float dx = 1.0f / (NUM_POINTS - 1);
    float dt = stability_factor * dx;

    float t = start_time;
    const float safety_factor = (1.0f - 1.0e-5f);

    int num_steps = 0;
    
    // Start measuring time
    struct timeval begin, end;
    gettimeofday(&begin, 0);

    while (t < safety_factor * stop_time) {
        // Make sure the last step does not go over the target time
        if (t + dt >= stop_time) {
            dt = stop_time - t;
        }

        float dtdxsq = (dt / dx) * (dt / dx);

        // Launch kernel to do the calculation
        wave_update(u, u_old, u_older, dtdxsq, N);

        // Swap u_old and u_older
        std::swap(u_old, u_older);

        // Swap u and u_old
        std::swap(u, u_old);

        // Print out diagnostics periodically
        if (num_steps % 100000 == 0) {
            std::cout << "Current integration time = " << t << "\n";
        }

        // Update t
        t += dt;
        ++num_steps;
    }
    
    gettimeofday(&end, 0);
    long seconds = end.tv_sec - begin.tv_sec;
    long microseconds = end.tv_usec - begin.tv_usec;
    double elapsed = seconds + microseconds*1e-6;
    
    printf("\n%d  %.3f seconds\n", NUM_POINTS, elapsed);

    // Clean up
    free(u_older);
    free(u_old);
    free(u);

    return 0;
}

In [ ]:
!g++ wave-omp.cpp -o wave-omp -fopenmp -O3

In [ ]:
!OMP_NUM_THREADS=12 ./wave-omp

| Program Version      | Execution Time (sec.)  | Speedup     |
| :---                 |    :----:              |        ---: |
| Serial               | 260.26                 | 1X           |
| OpenMP T=8           | 26.57                  | 10X        |
| OpenMP T=12          | 21.48                  | 12X        |
| OpenMP T=16          | 18.38                  | 14X        |
| OpenMP T=24          | 17.84                  | 15X        |   


## ⊗ CUDA

In [ ]:
%%writefile wave-cuda.cu
#include <iostream>
#include <limits>
#include <cstdio>
#include <cmath>
#include <stdio.h>
#include <stdlib.h>
#include <sys/time.h>

// Number of points in the overall spatial domain
#define NUM_POINTS 262144

__global__ void wave_update (float* u, const float* u_old, const float* u_older, float dtdxsq, int N)
{
    int idx = threadIdx.x + blockIdx.x * blockDim.x;

    if (idx > 0 && idx < N - 1) {

        float u_old_right = u_old[idx+1];
        float u_old_left = u_old[idx-1];

        u[idx] = 2.0f * u_old[idx] - u_older[idx] +
                 dtdxsq * (u_old_right - 2.0f * u_old[idx] + u_old_left);
    }
}

__global__ void initialize (float* u, int N)
{
    int idx = threadIdx.x + blockIdx.x * blockDim.x;

    if (idx < N) {
        u[idx] = std::sin(2.0f * M_PI * idx / static_cast<float>(NUM_POINTS - 1));
    }
}

int main(int argc, char **argv) {
    const int N = NUM_POINTS;

    // Allocate space for the grid data, and the temporary buffer
    // for the "old" and "older" data.
    float* u_older;
    float* u_old;
    float* u;

    cudaMalloc(&u_older, N * sizeof(float));
    cudaMalloc(&u_old, N * sizeof(float));
    cudaMalloc(&u, N * sizeof(float));

    // Initialize the data
    int threads_per_block = 256;
    int blocks = (N + threads_per_block - 1) / threads_per_block;

    initialize<<<blocks, threads_per_block>>>(u_older, N);
    initialize<<<blocks, threads_per_block>>>(u_old, N);
    initialize<<<blocks, threads_per_block>>>(u, N);
    cudaDeviceSynchronize();

    // Now iterate until we've completed a full period
    const float period = 1.0f;
    const float start_time = 0.0f;
    const float stop_time = period;

    // Maximum stable timestep is <= dx
    float stability_factor = 0.5f;
    float dx = 1.0f / (NUM_POINTS - 1);
    float dt = stability_factor * dx;

    float t = start_time;
    const float safety_factor = (1.0f - 1.0e-5f);

    int num_steps = 0;
    
    // Start measuring time
    struct timeval begin, end;
    gettimeofday(&begin, 0);

    while (t < safety_factor * stop_time) {
        // Make sure the last step does not go over the target time
        if (t + dt >= stop_time) {
            dt = stop_time - t;
        }

        float dtdxsq = (dt / dx) * (dt / dx);

        // Launch kernel to do the calculation
        wave_update<<<blocks, threads_per_block>>>(u, u_old, u_older, dtdxsq, N);

        // Swap u_old and u_older
        std::swap(u_old, u_older);

        // Swap u and u_old
        std::swap(u, u_old);

        // Print out diagnostics periodically
        if (num_steps % 100000 == 0) {
            std::cout << "Current integration time = " << t << "\n";
        }

        // Update t
        t += dt;
        ++num_steps;
    }

    gettimeofday(&end, 0);
    long seconds = end.tv_sec - begin.tv_sec;
    long microseconds = end.tv_usec - begin.tv_usec;
    double elapsed = seconds + microseconds*1e-6;
    
    printf("\n%d  %.3f seconds\n", NUM_POINTS, elapsed);
    
    // Clean up
    cudaFree(u_older);
    cudaFree(u_old);
    cudaFree(u);

    return 0;
}

In [ ]:
!nvcc wave-cuda.cu -o wave-cuda

In [ ]:
!./wave-cuda

| Program Version      | Execution Time (sec.)  | Speedup     |
| :---                 |    :----:              |        ---: |
| Serial               | 260.26                 | 1X           |
| OpenMP T=24          | 17.84                  | 15X        |   
| CUDA                 | 5.89                   | 44X        |   

## Next

Please continue to the next notebook: Please continue to the next notebook: [Final Exercise](4-finalExercise.ipynb).